# Evaluating AI with Haystack

by Bilge Yucel ([X](https://x.com/bilgeycl), [Linkedin](https://www.linkedin.com/in/bilge-yucel/))

In this cookbook, we walk through the [Evaluators](https://docs.haystack.deepset.ai/docs/evaluators) in Haystack, create an evaluation pipeline, streamline the evaluation with [`EvaluationHarness`](https://github.com/deepset-ai/haystack-experimental/tree/main/haystack_experimental/evaluation/harness) and try different Evaluation Frameworks like [Ragas](https://haystack.deepset.ai/integrations/ragas) and [FlowJudge](https://haystack.deepset.ai/integrations/flow-judge). 

📚 **Useful Resources:**
* [Article: Benchmarking Haystack Pipelines for Optimal Performance](https://haystack.deepset.ai/blog/benchmarking-haystack-pipelines)
* [Evaluation Walkthrough](https://haystack.deepset.ai/tutorials/guide_evaluation)
* [haystack-evaluation repo](https://github.com/deepset-ai/haystack-evaluation/tree/main)
* [EvaluationHarness (haystack-experimental)](https://github.com/deepset-ai/haystack-experimental/tree/main/haystack_experimental/evaluation/harness)
* [Evaluation tutorial](https://haystack.deepset.ai/tutorials/35_evaluating_rag_pipelines)
* [Evaluation Docs](https://docs.haystack.deepset.ai/docs/evaluation)

## 📺 Watch Along

<iframe width="560" height="315" src="https://www.youtube.com/embed/Dy-n_yC3Cto?si=LB0GdFP0VO-nJT-n" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [1]:
!pip install haystack-ai sentence-transformers>="3.0.0" pypdf

In [ ]:
!pip install ragas-haystack flow-judge[hf] # evaluation frameworks

## 1. Building your pipeline

### ARAGOG

This dataset is based on the paper [Advanced Retrieval Augmented Generation Output Grading (ARAGOG)](https://arxiv.org/pdf/2404.01037). It's a
collection of papers from ArXiv covering topics around Transformers and Large Language Models, all in PDF format.

The dataset contains:
- 13 PDF papers.
- 107 questions and answers generated with the assistance of GPT-4, and validated/corrected by humans.

We have:
- ground-truth answers
- questions

Get the dataset [here](https://github.com/deepset-ai/haystack-evaluation/blob/main/datasets/README.md)

### Indexing Pipeline

In [ ]:
import os

from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PyPDFToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

embedding_model="sentence-transformers/all-MiniLM-L6-v2"
document_store = InMemoryDocumentStore()

files_path = "/content/papers_for_questions" # <ENTER YOUR PATH HERE>
pipeline = Pipeline()
pipeline.add_component("converter", PyPDFToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_length=250, split_by="word"))  # default splitting by word
pipeline.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP))
pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(embedding_model))
pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "embedder")
pipeline.connect("embedder", "writer")
pdf_files = [files_path+"/"+f_name for f_name in os.listdir(files_path)]

pipeline.run({"converter": {"sources": pdf_files}})


In [4]:
document_store.count_documents()

690

### RAG

In [35]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY: ')

In [36]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder, AnswerBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers import InMemoryEmbeddingRetriever

template = """
    You have to answer the following question based on the given context information only.
    If the context is empty or just a '\\n' answer with None, example: "None".

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}

    Question: {{question}}
    Answer:
    """

basic_rag = Pipeline()
basic_rag.add_component("query_embedder", SentenceTransformersTextEmbedder(
    model=embedding_model, progress_bar=False
))
basic_rag.add_component("retriever", InMemoryEmbeddingRetriever(document_store))
basic_rag.add_component("prompt_builder", PromptBuilder(template=template))
basic_rag.add_component("generator", OpenAIGenerator(model="gpt-4o-mini"))

basic_rag.connect("query_embedder", "retriever.query_embedding")
basic_rag.connect("retriever", "prompt_builder.documents")
basic_rag.connect("prompt_builder", "generator")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

## 2. Human Evaluation

In [37]:
from typing import List, Tuple
import json

def read_question_answers() -> Tuple[List[str], List[str]]:
    with open("/content/eval_questions.json", "r") as f:
        data = json.load(f)
        questions = data["questions"]
        answers = data["ground_truths"]
    return questions, answers

all_questions, all_answers = read_question_answers()

In [38]:
print(len(all_questions))
print(len(all_answers))

107
107


In [39]:
questions = all_questions[:15]
answers = all_answers[:15]

In [40]:
index = 5
print(questions[index])
print(answers[index])
question = questions[index]

How were the questions for the multitask test sourced, and what was the criteria for their inclusion?
Questions were manually collected by graduate and undergraduate students from freely available online sources, including practice questions for standardized tests and undergraduate courses, ensuring a wide representation of difficulty levels and subjects.


In [41]:
basic_rag.run({"query_embedder":{"text":question}, "prompt_builder":{"question": question}})

{'generator': {'replies': ['The questions for the multitask test were manually collected by graduate and undergraduate students from freely available sources online. These sources included practice questions for tests such as the Graduate Record Examination and the United States Medical Licensing Examination, as well as questions designed for undergraduate courses and those for readers of Oxford University Press books. The criteria for inclusion were based on ensuring that the questions covered a range of subjects and difficulty levels, including specific tasks like "Elementary," "High School," "College," or "Professional," with each subject containing a minimum of 100 test examples.'],
  'meta': [{'model': 'gpt-4o-mini-2024-07-18',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 110,
     'prompt_tokens': 4559,
     'total_tokens': 4669,
     'completion_tokens_details': CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0),
     'prompt_tokens

## 3. Deciding on Metrics

* **Semantic Answer Similarity**: SASEvaluator compares the embedding of a generated answer against a ground-truth answer based on a common embedding model.
* **ContextRelevanceEvaluator** will assess the relevancy of the retrieved context to answer the query question
* **FaithfulnessEvaluator** evaluates whether the generated answer can be derived from the context


## 4. Building an Evaluation Pipeline

In [42]:
from haystack import Pipeline
from haystack.components.evaluators import ContextRelevanceEvaluator, FaithfulnessEvaluator, SASEvaluator

eval_pipeline = Pipeline()
eval_pipeline.add_component("context_relevance", ContextRelevanceEvaluator(raise_on_failure=False))
eval_pipeline.add_component("faithfulness", FaithfulnessEvaluator(raise_on_failure=False))
eval_pipeline.add_component("sas", SASEvaluator(model=embedding_model))

## 5. Running Evaluation

### Run the RAG Pipeline

In [43]:
predicted_answers = []
retrieved_context = []

for question in questions: # loops over 15 questions
  result = basic_rag.run({"query_embedder":{"text":question}, "prompt_builder":{"question": question}}, include_outputs_from={"retriever"})
  predicted_answers.append(result["generator"]["replies"][0])
  retrieved_context.append(result["retriever"]["documents"])

### Run the Evaluation

In [44]:
eval_pipeline_results = eval_pipeline.run(
    {
        "context_relevance": {"questions": questions, "contexts": retrieved_context},
        "faithfulness": {"questions": questions, "contexts": retrieved_context, "predicted_answers": predicted_answers},
        "sas": {"predicted_answers": predicted_answers, "ground_truth_answers": answers},
    }
)

results = {
    "context_relevance": eval_pipeline_results['context_relevance'],
    "faithfulness": eval_pipeline_results['faithfulness'],
    "sas": eval_pipeline_results['sas']
}

100%|██████████| 15/15 [00:35<00:00,  2.37s/it]


## 6. Analyzing Results

[EvaluationRunResult](https://docs.haystack.deepset.ai/reference/evaluation-api#evaluationrunresult)

In [45]:
from haystack.evaluation import EvaluationRunResult

inputs = {
    'questions': questions,
    'contexts': retrieved_context,
    'true_answers': answers,
    'predicted_answers': predicted_answers
}
run_name="rag_eval"
eval_results = EvaluationRunResult(run_name=run_name, inputs=inputs, results=results)
eval_results.score_report()

,metrics,score
0,context_relevance,0.200000
1,faithfulness,0.611111
2,sas,0.546086


In [ ]:
eval_results.to_pandas()

In [ ]:
index = 2
print(eval_pipeline_results['context_relevance']["individual_scores"][index], "\nQuestion:", questions[index],"\nTrue Answer:", answers[index], "\nAnswer:", predicted_answers[index])
print("".join([doc.content for doc in retrieved_context[index]]))

## Evaluation Harness (Step 4, 5, and 6)

* Runs the RAG pipeline
* Runs the evaluation

> Try `EvaluationHarness` and give us feedback [on Github](https://github.com/deepset-ai/haystack-experimental/discussions/74)

In [ ]:
from haystack_experimental.evaluation.harness.rag import (
    DefaultRAGArchitecture,
    RAGEvaluationHarness,
    RAGEvaluationMetric,
    RAGEvaluationInput
)

pipeline_eval_harness = RAGEvaluationHarness(
    rag_pipeline = basic_rag,
    rag_components=DefaultRAGArchitecture.GENERATION_WITH_EMBEDDING_RETRIEVAL, # query_embedder, retriever, prompt_builder, generator
    metrics={
        RAGEvaluationMetric.SEMANTIC_ANSWER_SIMILARITY,
        RAGEvaluationMetric.FAITHFULNESS,
        RAGEvaluationMetric.CONTEXT_RELEVANCE,
    }
)

eval_harness_input = RAGEvaluationInput(
    queries=questions,
    ground_truth_answers=answers,
    rag_pipeline_inputs={
        "prompt_builder": {"question": list(questions)},
    },
)

harness_eval_run= pipeline_eval_harness.run(inputs=eval_harness_input, run_name=run_name)

In [49]:
harness_eval_run.results.score_report()

,metrics,score
0,metric_context_relevance,0.266667
1,metric_sas,0.537721
2,metric_faithfulness,0.747778


Override some parameter

In [ ]:
from haystack_experimental.evaluation.harness.rag import RAGEvaluationOverrides

overrides = RAGEvaluationOverrides(rag_pipeline={
    "generator": {"model": "gpt-4"},
})

harness_eval_run_gpt4 = pipeline_eval_harness.run(inputs=eval_harness_input, run_name="harness_eval_run_gpt4", overrides=overrides)

In [51]:
harness_eval_run_gpt4.results.score_report()

,metrics,score
0,metric_context_relevance,0.266667
1,metric_sas,0.654073
2,metric_faithfulness,0.796429


In [ ]:
harness_eval_run.results.comparative_individual_scores_report(harness_eval_run_gpt4.results)

In [ ]:
overrides = RAGEvaluationOverrides(rag_pipeline={
    "retriever": {"top_k": 2},
})

harness_eval_run_topk10 = pipeline_eval_harness.run(inputs=eval_harness_input, run_name="harness_eval_run_topk10", overrides=overrides)

Executing RAG pipeline...


100%|██████████| 30/30 [01:50<00:00,  3.67s/it]


Executing evaluation pipeline...


100%|██████████| 30/30 [00:26<00:00,  1.12it/s]


In [ ]:
harness_eval_run_topk10.results.score_report()

,metrics,score
0,metric_sas,0.574303
1,metric_faithfulness,0.780000
2,metric_context_relevance,0.400000


## Evaluation Frameworks

* [RagasEvaluator](https://docs.haystack.deepset.ai/docs/ragasevaluator)
* [FlowJudge](https://haystack.deepset.ai/integrations/flow-judge)

In [ ]:
from flow_judge.integrations.haystack import HaystackFlowJudge
from flow_judge.metrics.presets import RESPONSE_FAITHFULNESS_5POINT
from flow_judge import Hf

model = Hf(flash_attn=False)

flow_judge_evaluator = HaystackFlowJudge(
    metric=RESPONSE_FAITHFULNESS_5POINT,
    model=model,
    progress_bar=True,
    raise_on_failure=True,
    save_results=True,
    fail_on_parse_error=False
)

In [ ]:
from haystack_integrations.components.evaluators.ragas import RagasEvaluator, RagasMetric

ragas_evaluator= RagasEvaluator(
    metric=RagasMetric.FAITHFULNESS
)

In [28]:
str_fj_retrieved_context = []
for context in retrieved_context:
  str_context = [doc.content for doc in context]
  str_fj_retrieved_context.append(" ".join(str_context)) # ["", "", ...]

In [29]:
str_retrieved_context = []
for context in retrieved_context:
  str_context = [doc.content for doc in context]
  str_retrieved_context.append(str_context) # [["", ""]]

In [31]:
from haystack import Pipeline

integration_eval_pipeline = Pipeline()
integration_eval_pipeline.add_component("ragas_evaluator", ragas_evaluator)
integration_eval_pipeline.add_component("flow_judge_evaluator", flow_judge_evaluator)

eval_framework_pipeline_results = integration_eval_pipeline.run(
    {
        "ragas_evaluator": {"questions": questions, "contexts": str_retrieved_context, "responses":predicted_answers},
        "flow_judge_evaluator": {"query": questions, "context": str_fj_retrieved_context, "response": predicted_answers},
    }
)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

2024-10-30T15:42:26.834829Z [warning  ] No statements were generated from the answer. lineno=232 module=ragas.metrics._faithfulness
2024-10-30T15:42:27.247699Z [warning  ] No statements were generated from the answer. lineno=232 module=ragas.metrics._faithfulness
2024-10-30T15:42:27.338863Z [warning  ] No statements were generated from the answer. lineno=232 module=ragas.metrics._faithfulness
Processing batches: 100%|██████████| 10/10 [03:32<00:00, 21.23s/it]


In [34]:
eval_framework_pipeline_results

{'ragas_evaluator': {'results': [[{'name': 'faithfulness', 'score': 0.5}],
   [{'name': 'faithfulness', 'score': 1.0}],
   [{'name': 'faithfulness', 'score': 1.0}],
   [{'name': 'faithfulness', 'score': nan}],
   [{'name': 'faithfulness', 'score': nan}],
   [{'name': 'faithfulness', 'score': 0.9090909090909091}],
   [{'name': 'faithfulness', 'score': 1.0}],
   [{'name': 'faithfulness', 'score': 1.0}],
   [{'name': 'faithfulness', 'score': 1.0}],
   [{'name': 'faithfulness', 'score': nan}]]},
 'flow_judge_evaluator': {'results': [{'feedback': "The response provided is highly consistent with the given context. The context explicitly mentions that BERT is pre-trained on two tasks: the masked language model (MLM) task and the next sentence prediction (NSP) task. The response accurately identifies these two tasks as the main pre-training tasks for BERT, directly reflecting the information provided in the context. There are no hallucinated or fabricated details in the response, and all the i